In [115]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
import os
import json


def transform_data(data, start_idx=0, skip_relations=[]):
    new_data = []
    identity_counter = start_idx

    for conversation, triples in data:
        triples_text = []
        for triple in triples:
            x, y, r = triple["x"], triple["y"], triple["r"][0].split(':')[-1] if triple["r"] else None
            if r and (not r in skip_relations):
                triples_text.append({
                    "x": x,
                    "r": r,
                    "y": y
                })
                

        
        conversation_text = "" + str(json.dumps(triples_text))
        output_text = "\n".join(conversation)

        conversation_entry = {
            "id": f"identity_{identity_counter}",
            "conversations": [
                {
                    "from": "human",
                    "value": output_text
                    
                },
                {
                    "from": "gpt",
                    "value": conversation_text
                }
            ]
        }
        
        identity_counter += 1
        
        if triples_text == []:
            continue
        
        new_data.append(conversation_entry)

    return new_data

In [120]:
all_relations = {"positive_impression", "negative_impression", "acquaintance", "alumni", "boss", "subordinate", "client", "dates", "friends", "girl/boyfriend", "neighbor", "roommate", "children", "other_family", "parents", "siblings", "spouse", "place_of_residence", "visited_place", "origin", "employee_or_member_of", "schools_attended", "works", "age", "date_of_birth", "major", "place_of_work", "title", "alternate_names", "pet", "residents_of_place", "visitors_of_place", "employees_or_members", "students", "unanswerable"}
allowed_relations = {"positive_impression", "negative_impression", "acquaintance", "alumni", "boss", "subordinate", "client", "dates", "friends", "girl/boyfriend", "neighbor", "roommate", "children", "other_family", "parents", "siblings", "spouse", "place_of_residence", "visited_place", "origin", "employee_or_member_of", "schools_attended", "works", "age", "date_of_birth", "major", "place_of_work", "title", "alternate_names", "pet", "residents_of_place", "visitors_of_place", "employees_or_members", "students", "unanswerable"}
# allowed_relations = {"acquaintance", "children", "other_family", "parents", "siblings", "spouse", "place_of_residence", "visited_place", "pet", "residents_of_place", "visitors_of_place"}
skip_relations = [r for r in all_relations if not r in allowed_relations]
print("relations to keep=",len(allowed_relations))

relations to keep= 35


In [121]:
import os
import json

# Paths
INPUT_DIR = "/home/murilo/RelNetCare/data/raw/dialog-re"
OUTPUT_DIR = "/home/murilo/RelNetCare/data/processed/dialog-re-llama"
file_sets = [['train', 'dev'], ['test']]

for files in file_sets:
    output_dataset_name = f"dialog-re-llama-{len(allowed_relations)}cls-{'-'.join(files)}"
    last_data_idx = 0
    new_format = []

    # Check and create directory if not exists
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    for f in files:
        input_data_path = os.path.join(INPUT_DIR, f'{f}.json')
        
        # Ensure the file exists before attempting to open it
        if os.path.exists(input_data_path):
            with open(input_data_path, encoding='utf8') as fp:
                data = json.load(fp)
            
            new_format.extend(transform_data(data, start_idx=last_data_idx, skip_relations=skip_relations))
            last_data_idx = len(data)

    output_data_path = os.path.join(OUTPUT_DIR, f'{output_dataset_name}.json')


    # Here, we'll write to the file instead of reading it.
    # Assuming you want to save the newly formatted data.
    with open(output_data_path, 'w', encoding='utf8') as fp:
        json.dump(new_format, fp)

    print(files,len(new_format))

['train', 'dev'] 1431
['test'] 357


In [103]:
x = "[{\"x\": \"Speaker 1\", \"r\": \"spouse\", \"y\": \"Carol\"}, {\"x\": \"Carol\", \"r\": \"spouse\", \"y\": \"Speaker 1\"}]"
 
 
# Evaluate the string to get a list of strings
x_list = eval(x)

# Convert each string in the list into a dictionary
# dicts = [json.loads(item) for item in x_list]

# print(dicts)

In [37]:
input_data_path = os.path.join(INPUT_DIR, f'test.json')

with open(input_data_path, encoding='utf8') as fp:
    data = json.load(fp)

In [41]:
idx = 0
tmp = transform_data(data)[idx]

In [47]:
tmp['conversations'][0]['value']
tmp['conversations'][1]['value']

"[{'x': 'Speaker 1', 'r': 'per:age', 'y': '50'}, {'x': 'Speaker 1', 'r': 'per:place_of_residence', 'y': 'Montauk'}, {'x': 'Speaker 1', 'r': 'per:alumni', 'y': 'Frank'}, {'x': 'Frank', 'r': 'per:alumni', 'y': 'Speaker 1'}, {'x': 'Montauk', 'r': 'gpe:residents_of_place', 'y': 'Speaker 1'}, {'x': 'Speaker 7', 'r': 'unanswerable', 'y': 'pal'}, {'x': 'Speaker 3', 'r': 'unanswerable', 'y': 'pal'}, {'x': 'Speaker 3', 'r': 'unanswerable', 'y': 'Dad'}, {'x': 'Speaker 2', 'r': 'unanswerable', 'y': 'pal'}]"

In [50]:
gt = eval("[{'x': 'Speaker 1', 'r': 'per:age', 'y': '50'}, {'x': 'Speaker 1', 'r': 'per:place_of_residence', 'y': 'Montauk'}, {'x': 'Speaker 1', 'r': 'per:alumni', 'y': 'Frank'}, {'x': 'Frank', 'r': 'per:alumni', 'y': 'Speaker 1'}, {'x': 'Montauk', 'r': 'gpe:residents_of_place', 'y': 'Speaker 1'}, {'x': 'Speaker 7', 'r': 'unanswerable', 'y': 'pal'}, {'x': 'Speaker 3', 'r': 'unanswerable', 'y': 'pal'}, {'x': 'Speaker 3', 'r': 'unanswerable', 'y': 'Dad'}, {'x': 'Speaker 2', 'r': 'unanswerable', 'y': 'pal'}]")

In [52]:
gt

[{'x': 'Speaker 1', 'r': 'per:age', 'y': '50'},
 {'x': 'Speaker 1', 'r': 'per:place_of_residence', 'y': 'Montauk'},
 {'x': 'Speaker 1', 'r': 'per:alumni', 'y': 'Frank'},
 {'x': 'Frank', 'r': 'per:alumni', 'y': 'Speaker 1'},
 {'x': 'Montauk', 'r': 'gpe:residents_of_place', 'y': 'Speaker 1'},
 {'x': 'Speaker 7', 'r': 'unanswerable', 'y': 'pal'},
 {'x': 'Speaker 3', 'r': 'unanswerable', 'y': 'pal'},
 {'x': 'Speaker 3', 'r': 'unanswerable', 'y': 'Dad'},
 {'x': 'Speaker 2', 'r': 'unanswerable', 'y': 'pal'}]

In [57]:
prediction = "[{‘x’: ‘Speaker 6’, ‘r’: ‘per:alternate_names’, ‘y’: ‘Phoebe’}, {‘x’: ‘Speaker 6’, ‘r’: ‘per:alternate_names’, ‘y’:‘my pal Phoebe’}, {‘x’: ‘Speaker 1’, ‘r’: ‘per:parents’, ‘y’: ‘Speaker 1’s parents’}, {‘x’: ‘Speaker 4’, ‘r’: ‘per:friends’, ‘y’: ‘Speaker 6’}, {‘x’: ‘Speaker 1’s parents’, ‘r’: ‘per:children’, ‘y’: ‘Speaker 1’}, {‘x’: ‘Speaker 7’, ‘r’: ‘unanswerable’, ‘y’:‘my pal Phoebe’}]"

In [61]:
[{'x': 'Speaker 6', 'r': 'per:alternate_names', 'y': 'Phoebe'},
 {'x': 'Speaker 6', 'r': 'per:alternate_names', 'y':'my pal Phoebe'},
 {'x': 'Speaker 1', 'r': 'per:parents', 'y': 'Speaker 1\'s parents'}, 
  {'x': 'Speaker 4', 'r': 'per:friends', 'y': 'Speaker 6'},
  {'x': 'Speaker 1\'s parents', 'r': 'per:children', 'y': 'Speaker 1'},
  {'x': 'Speaker 7', 'r': 'unanswerable', 'y':'my pal Phoebe'}]


[{'x': 'Speaker 6', 'r': 'per:alternate_names', 'y': 'Phoebe'},
 {'x': 'Speaker 6', 'r': 'per:alternate_names', 'y': 'my pal Phoebe'},
 {'x': 'Speaker 1', 'r': 'per:parents', 'y': "Speaker 1's parents"},
 {'x': 'Speaker 4', 'r': 'per:friends', 'y': 'Speaker 6'},
 {'x': "Speaker 1's parents", 'r': 'per:children', 'y': 'Speaker 1'},
 {'x': 'Speaker 7', 'r': 'unanswerable', 'y': 'my pal Phoebe'}]

In [62]:
gt

[{'x': 'Speaker 1', 'r': 'per:age', 'y': '50'},
 {'x': 'Speaker 1', 'r': 'per:place_of_residence', 'y': 'Montauk'},
 {'x': 'Speaker 1', 'r': 'per:alumni', 'y': 'Frank'},
 {'x': 'Frank', 'r': 'per:alumni', 'y': 'Speaker 1'},
 {'x': 'Montauk', 'r': 'gpe:residents_of_place', 'y': 'Speaker 1'},
 {'x': 'Speaker 7', 'r': 'unanswerable', 'y': 'pal'},
 {'x': 'Speaker 3', 'r': 'unanswerable', 'y': 'pal'},
 {'x': 'Speaker 3', 'r': 'unanswerable', 'y': 'Dad'},
 {'x': 'Speaker 2', 'r': 'unanswerable', 'y': 'pal'}]